# Satellite Container Analysis for Trading Signals

This notebook implements the satellite image analysis system described in the research to:
1. Collect satellite imagery from major ports
2. Use deep learning to count shipping containers
3. Generate trading signals from container volume changes

In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ee
import torch
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import requests
from PIL import Image
from io import BytesIO

# Import our new Google Earth Engine collector
from gee_collector import GoogleEarthCollector

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo26n-obb.pt")  # load a pretrained model (recommended for training)
results = model.train(data="dota8.yaml", epochs=100, imgsz=640)

In [ ]:
# Load configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Initialize Google Earth Engine with OAuth authentication
# Note: Run setup_gee.py first if this is your first time
try:
    ee.Initialize(project=config['GoogleOAuth']['ProjectId'])
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print("⚠️  Earth Engine not authenticated yet")
    print("Please run: python setup_gee.py")
    print("\nAttempting OAuth authentication now...")
    ee.Authenticate()
    ee.Initialize(project=config['GoogleOAuth']['ProjectId'])
    print("✓ Earth Engine authenticated and initialized")

ports = config['ports']
print(f"\n✓ Loaded {len(ports)} major ports for analysis:")
for port in ports:
    print(f"  • {port['name']:15} ({port['country']})")

## Port Locations

Let's visualize the major ports we're analyzing:

In [ ]:
# Display port information
import plotly.graph_objects as go

print("Major Ports for Container Analysis:")
print("=" * 70)
for i, port in enumerate(ports, 1):
    print(f"{i}. {port['name']:15} | {port['country']:15} | Lat: {port['lat']:7.4f}, Lon: {port['lon']:8.4f}")
print("=" * 70)

# Create world map with port locations
fig = go.Figure(data=go.Scattergeo(
    lon=[p['lon'] for p in ports],
    lat=[p['lat'] for p in ports],
    text=[f"{p['name']}<br>{p['country']}" for p in ports],
    mode='markers+text',
    marker=dict(
        size=15,
        color='red',
        line=dict(width=2, color='white')
    ),
    textposition="top center",
    textfont=dict(size=12, color='black', family='Arial Black')
))

fig.update_layout(
    title='Major Ports for Container Analysis',
    geo=dict(
        projection_type='natural earth',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
        showcountries=True,
        countrycolor='rgb(204, 204, 204)',
    ),
    height=500,
)

fig.show()

In [ ]:
# Visualize downloaded images
def show_port_images(df, max_images=6):
    """Display grid of downloaded satellite images"""
    if df.empty:
        print("No images to display")
        return
    
    # Take first max_images
    sample_df = df.head(max_images)
    
    # Create subplot grid
    n_images = len(sample_df)
    cols = 3
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, row) in enumerate(sample_df.iterrows()):
        ax = axes[idx // cols, idx % cols]
        
        # Load and display image
        img = Image.open(row['filepath'])
        ax.imshow(img)
        ax.set_title(f"{row['port']}\n{row['date']}\nCloud: {row['cloud_cover']:.1f}%", 
                     fontsize=10, fontweight='bold')
        ax.axis('off')
    
    # Hide empty subplots
    for idx in range(n_images, rows * cols):
        axes[idx // cols, idx % cols].axis('off')
    
    plt.tight_layout()
    plt.show()

# Display images from all ports download
if 'df_all_ports' in locals() and not df_all_ports.empty:
    print("Sample of downloaded satellite images:")
    show_port_images(df_all_ports, max_images=6)
elif 'df_shanghai' in locals() and not df_shanghai.empty:
    print("Shanghai port satellite images:")
    show_port_images(df_shanghai, max_images=6)
else:
    print("⚠️  No images available. Run download cells above first.")

### Visualize Downloaded Images

Let's preview the downloaded satellite images:

In [ ]:
# Download images for all ports
print("Downloading images for all ports...")
print("This may take several minutes...\n")

df_all_ports = collector.collect_port_images(
    days_back=30,          # Last 30 days
    max_images=3,          # 3 images per port
    max_cloud=10,          # Max 10% cloud cover
    source='sentinel2'     # Sentinel-2 (10m resolution)
)

# Display summary
if not df_all_ports.empty:
    print("\n" + "="*70)
    print("✓ DOWNLOAD COMPLETE!")
    print("="*70)
    
    # Summary by port
    summary = df_all_ports.groupby('port').agg({
        'date': 'count',
        'cloud_cover': 'mean'
    }).rename(columns={'date': 'images', 'cloud_cover': 'avg_cloud%'})
    
    print("\nSummary by port:")
    display(summary)
    
    print(f"\nTotal images downloaded: {len(df_all_ports)}")
    print(f"Date range: {df_all_ports['date'].min()} to {df_all_ports['date'].max()}")
    print(f"Images saved to: data/images/")
    print(f"Metadata saved to: data/results/image_metadata.csv")
    
    # Show sample of downloaded files
    print("\nSample files:")
    display(df_all_ports[['port', 'country', 'date', 'cloud_cover', 'filepath']].head(10))
else:
    print("\n⚠️  No images downloaded for any port")
    print("Check date range and cloud cover settings")

### Option 2: Download Images for All Ports

In [ ]:
# Test download for Shanghai port
print("Downloading images for Shanghai...")
print("="*70)

df_shanghai = collector.collect_port_images(
    port_name='Shanghai',
    days_back=60,          # Look back 60 days
    max_images=5,          # Download up to 5 images
    max_cloud=15,          # Max 15% cloud cover
    source='sentinel2'     # Use Sentinel-2 (10m resolution)
)

# Display results
if not df_shanghai.empty:
    print("\n✓ Download complete!")
    print("\nDownloaded images:")
    display(df_shanghai[['port', 'date', 'cloud_cover', 'resolution_m', 'filepath']])
    
    # Show image count by date
    print(f"\nTotal images: {len(df_shanghai)}")
    print(f"Date range: {df_shanghai['date'].min()} to {df_shanghai['date'].max()}")
    print(f"Average cloud cover: {df_shanghai['cloud_cover'].mean():.1f}%")
else:
    print("\n⚠️  No images found. Try:")
    print("  • Increasing days_back (e.g., 90)")
    print("  • Increasing max_cloud (e.g., 20)")
    print("  • Using a different port")

### Option 1: Download Images for a Single Port (Quick Test)

In [ ]:
# Initialize the Google Earth Engine collector
# authenticate=False because we already did OAuth authentication above
collector = GoogleEarthCollector(authenticate=False)

print("✓ Google Earth Engine collector ready!")
print("\nAvailable features:")
print("  • Sentinel-2 imagery (10m resolution)")
print("  • Landsat 8/9 imagery (30m resolution)")
print("  • Automatic cloud filtering")
print("  • Metadata tracking")

## Download Satellite Images with Google Earth Engine

Now we'll use the new Google Earth Engine collector to download high-quality satellite imagery.

In [ ]:
## Legacy Data Collector (Optional)

The old SatelliteDataCollector is kept for reference, but we now use GoogleEarthCollector above.
You can skip this cell.

In [ ]:
class ContainerDetector:
    def __init__(self, model_type='dota'):
        """
        Initialize container detector with satellite-specific models
        
        Args:
            model_type: 'dota' (aerial detection) or 'yolo-coco' (ground-level)
        """
        self.model_type = model_type
        
        if model_type == 'dota':
            # Try to load DOTA-trained YOLO model for aerial imagery
            try:
                # First try custom DOTA model if available
                self.model = YOLO('yolov8n-dota.pt')
                print("✓ Loaded YOLOv8 trained on DOTA dataset")
            except:
                try:
                    # Try DOTAv2 model
                    self.model = YOLO('yolov8n-dotav2.pt')
                    print("✓ Loaded YOLOv8 trained on DOTAv2 dataset")
                except:
                    # Fall back to downloading from Ultralytics hub or training
                    print("⚠️  DOTA model not found locally")
                    print("   Attempting to use YOLOv8 with satellite-optimized settings...")
                    self.model = YOLO('yolov8n.pt')
                    self.model_type = 'yolo-optimized'
        else:
            # Standard COCO-trained model
            self.model = YOLO('yolov8n.pt')
            print("✓ Loaded standard YOLOv8 (COCO dataset)")
    
    def preprocess_image(self, image_path):
        """Preprocess satellite image for container detection"""
        img = cv2.imread(str(image_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # For satellite imagery, enhance contrast
        if self.model_type in ['dota', 'yolo-optimized']:
            # Convert to LAB color space for better contrast
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            
            # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
            l = clahe.apply(l)
            
            # Merge and convert back
            enhanced = cv2.merge([l, a, b])
            img = cv2.cvtColor(enhanced, cv2.COLOR_LAB2RGB)
        
        return img
    
    def detect_containers(self, image, show_all_classes=False):
        """Detect and count containers in satellite image"""
        
        if self.model_type == 'dota':
            # DOTA dataset classes for aerial imagery
            # Classes include: ship, harbor, vehicle, plane, storage-tank, etc.
            conf_threshold = 0.25
            container_classes = ['ship', 'harbor', 'large-vehicle', 'small-vehicle', 'storage-tank']
        else:
            # Standard COCO with optimized settings for satellite
            conf_threshold = 0.05  # Very low threshold for tiny objects
            container_classes = [2, 5, 7, 8]  # car, bus, truck, boat
        
        results = self.model(image, conf=conf_threshold, verbose=False, imgsz=1024)
        
        containers = []
        all_detections = []
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    cls = int(box.cls)
                    conf = float(box.conf)
                    class_name = self.model.names[cls]
                    
                    # Track all detections for analysis
                    all_detections.append({
                        'class_id': cls,
                        'class_name': class_name,
                        'confidence': conf
                    })
                    
                    # Filter for container-like objects
                    is_container = False
                    if self.model_type == 'dota':
                        is_container = class_name in container_classes
                    else:
                        is_container = cls in container_classes
                    
                    if is_container and conf > conf_threshold:
                        containers.append({
                            'bbox': box.xyxy.cpu().numpy(),
                            'confidence': conf,
                            'class': cls,
                            'class_name': class_name
                        })
        
        if show_all_classes:
            return len(containers), containers, all_detections
        return len(containers), containers
    
    def analyze_port_activity(self, image_dir, port_name):
        """Analyze container activity for a specific port"""
        image_files = list(Path(image_dir).glob('*.jp*g'))
        results = []
        
        for img_path in image_files:
            img = self.preprocess_image(img_path)
            count, detections = self.detect_containers(img)
            
            results.append({
                'port': port_name,
                'image': img_path.name,
                'container_count': count,
                'timestamp': datetime.now()
            })
        
        return pd.DataFrame(results)

In [ ]:
class TradingSignalGenerator:
    def __init__(self):
        self.container_data = pd.DataFrame()
        
    def load_container_data(self, data):
        """Load container count data"""
        self.container_data = data
        
    def calculate_signals(self):
        """Generate trading signals from container volume changes"""
        if self.container_data.empty:
            return pd.DataFrame()
        
        # Group by port and calculate rolling statistics
        signals = []
        
        for port in self.container_data['port'].unique():
            port_data = self.container_data[self.container_data['port'] == port].copy()
            port_data = port_data.sort_values('timestamp')
            
            # Calculate moving averages and changes
            port_data['ma_7'] = port_data['container_count'].rolling(7).mean()
            port_data['ma_30'] = port_data['container_count'].rolling(30).mean()
            port_data['pct_change'] = port_data['container_count'].pct_change()
            
            # Generate signals
            port_data['signal'] = 0
            port_data.loc[port_data['ma_7'] > port_data['ma_30'], 'signal'] = 1  # Bullish
            port_data.loc[port_data['ma_7'] < port_data['ma_30'], 'signal'] = -1  # Bearish
            
            # Warning signal for extremely high volumes
            high_threshold = port_data['container_count'].quantile(0.95)
            port_data['warning'] = port_data['container_count'] > high_threshold
            
            signals.append(port_data)
        
        return pd.concat(signals, ignore_index=True)
    
    def generate_global_signal(self, port_signals):
        """Generate global trading signal from all ports"""
        global_signal = port_signals.groupby('timestamp').agg({
            'container_count': 'sum',
            'signal': 'mean',
            'warning': 'any'
        }).reset_index()
        
        global_signal['global_signal'] = np.where(
            global_signal['signal'] > 0.2, 1,
            np.where(global_signal['signal'] < -0.2, -1, 0)
        )
        
        return global_signal

## Install DOTA-Trained Model

Three options to get satellite-specific detection:

In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

shanghai = ports[0]  # Shanghai port
print(f"\nSearching for imagery of {shanghai['name']} port...")
print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

# Get collection info
collection, roi = collector.get_port_imagery(
    shanghai, 
    start_date.strftime('%Y-%m-%d'),
    end_date.strftime('%Y-%m-%d')
)

count = collection.size().getInfo()
print(f"✓ Found {count} Sentinel-2 images with <20% cloud cover")

# Download more images (increased from 3 to 20)
if count > 0:
    print("\nDownloading images...")
    downloaded = collector.collect_port_images(shanghai, 
                                                start_date.strftime('%Y-%m-%d'),
                                                end_date.strftime('%Y-%m-%d'),
                                                max_images=1000)  # Download up to 1000 images
    print(f"\n✓ Downloaded {len(downloaded)} images")

## Sample Image Download and Visualization

Now let's download actual satellite images and visualize them:

In [ ]:
# Visualize ALL downloaded images
if 'downloaded' in dir() and len(downloaded) > 0:
    print(f"\n{'='*70}")
    print(f"Downloaded {len(downloaded)} satellite images for {shanghai['name']}")
    print(f"{'='*70}\n")
    
    # Calculate grid dimensions for all images
    n_images = len(downloaded)
    n_cols = min(5, n_images)  # Max 5 columns
    n_rows = (n_images + n_cols - 1) // n_cols  # Ceiling division
    
    # Create grid of subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4))
    
    # Flatten axes array for easy iteration
    if n_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_rows > 1 else axes
    
    # Display all images
    for idx, img_info in enumerate(downloaded):
        img = Image.open(img_info['filepath'])
        axes[idx].imshow(img)
        axes[idx].set_title(f"{img_info['port']}\n{img_info['date']}", 
                           fontsize=10, fontweight='bold')
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_images, len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f'Sentinel-2 Satellite Imagery - {shanghai["name"]} Port ({n_images} images)', 
                 fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Print detailed summary
    print(f"\n Image Summary:")
    print(f"  • Total images: {n_images}")
    print(f"  • Date range: {downloaded[0]['date']} to {downloaded[-1]['date']}")
    print(f"\n Image Details:")
    for i, img_info in enumerate(downloaded, 1):
        print(f"  {i:2d}. {img_info['date']} - {img_info['filepath'].name}")
else:
    print(" No images downloaded yet. Run the previous cell first.")

In [ ]:
# Inference on all downloaded satellite images using trained OBB model
from ultralytics import YOLO
from pathlib import Path
import pandas as pd
import polars as pl  # Add polars for DataFrame conversion if needed

# Load the best trained OBB model
model = YOLO("runs/obb/train/weights/best.pt")

# Directory containing satellite images (update if needed)
image_dir = Path("data/images")
image_files = list(image_dir.glob("*.jpg"))

results_list = []

for img_path in image_files:
    results = model(str(img_path))
    df_polars = results[0].to_df()
    # Convert Polars DataFrame to pandas DataFrame if possible
    df = df_polars.to_pandas() if hasattr(df_polars, "to_pandas") else pd.DataFrame(df_polars)
    df["image"] = img_path.name
    results_list.append(df)
    print(f"✓ Processed {img_path.name} - {len(df)} detections")

# Combine all results into a single DataFrame
if results_list:
    all_detections = pd.concat(results_list, ignore_index=True)
    print("\nDetection results for all images:")
    print(all_detections.head())
    # Optionally, save to CSV
    Path("data/results").mkdir(parents=True, exist_ok=True)
    all_detections.to_csv("data/results/obb_detections.csv", index=False)
    print("\nAll detection results saved to data/results/obb_detections.csv")
else:
    print("No images found or no detections made.")

## Container Detection on Downloaded Images

Apply YOLO model to detect containers in the satellite images:

In [ ]:
# Run container detection on downloaded images
if 'downloaded' in dir() and len(downloaded) > 0:
    print("Running YOLO container detection...")
    print("⚠️  NOTE: YOLO is trained on ground photos, not satellite imagery!")
    print("=" * 70)
    
    detection_results = []
    all_classes_found = {}
    
    for img_info in downloaded:
        # Load and preprocess image
        img = detector.preprocess_image(img_info['filepath'])
        
        # Detect containers and get all detections
        count, detections, all_dets = detector.detect_containers(img, show_all_classes=True)
        
        # Track what classes were found
        for det in all_dets:
            class_name = det['class_name']
            if class_name not in all_classes_found:
                all_classes_found[class_name] = 0
            all_classes_found[class_name] += 1
        
        detection_results.append({
            'port': img_info['port'],
            'date': img_info['date'],
            'filepath': img_info['filepath'],
            'container_count': count,
            'detections': detections,
            'all_detections': all_dets
        })
        
        print(f"✓ {img_info['date']}: {count:3d} vehicles | {len(all_dets):3d} total objects")
    
    print("=" * 70)
    
    # Show what YOLO actually detected
    print(f"\n🔍 All Object Classes Detected (across all {len(downloaded)} images):")
    if all_classes_found:
        for class_name, count in sorted(all_classes_found.items(), key=lambda x: x[1], reverse=True):
            print(f"  • {class_name:20s}: {count:4d} instances")
    else:
        print("  ⚠️  No objects detected at all!")
        print("  This is because:")
        print("    - Satellite images are taken from 700km altitude")
        print("    - YOLO expects ground-level photos (0-100m)")
        print("    - Objects are too small (< 5 pixels)")
    
    # Visualize detections on image with most detections
    if detection_results:
        # Find image with most detections
        best_result = max(detection_results, key=lambda x: len(x['all_detections']))
        
        img = cv2.imread(str(best_result['filepath']))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Draw ALL detections (not just vehicles)
        for det in best_result['all_detections'][:50]:  # Limit to 50 for clarity
            # Find bounding boxes from the model results
            pass
        
        # Draw vehicle detections with boxes
        for det in best_result['detections']:
            bbox = det['bbox'][0]
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            label = f"{det['class_name']} {det['confidence']:.2f}"
            cv2.putText(img, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display
        plt.figure(figsize=(14, 10))
        plt.imshow(img)
        plt.title(f"Best Detection Result - {best_result['port']} ({best_result['date']})\n{best_result['container_count']} vehicles | {len(best_result['all_detections'])} total objects", 
                 fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        total_vehicles = sum(r['container_count'] for r in detection_results)
        total_objects = sum(len(r['all_detections']) for r in detection_results)
        print(f"\n📊 Detection Summary:")
        print(f"  • Total vehicles detected: {total_vehicles} (across {len(detection_results)} images)")
        print(f"  • Total objects detected: {total_objects}")
        print(f"  • Average per image: {total_vehicles/len(detection_results):.1f} vehicles, {total_objects/len(detection_results):.1f} objects")
        
        if total_vehicles == 0:
            print(f"\n💡 Recommendation:")
            print(f"  For satellite imagery, you need:")
            print(f"  1. Models trained on aerial/satellite data (not COCO)")
            print(f"  2. Higher resolution images (4096x4096 or larger)")
            print(f"  3. Specialized container detection models")
            print(f"  4. Or use synthetic data for demonstration (next cell)")
else:
    print("⚠️  No images available for detection. Download images first.")

In [ ]:
# Simulate container detection results (replace with actual image analysis)
print("Generating synthetic container count data for demonstration...")
print(f"Ports included: {', '.join([p['name'] for p in ports])}\n")

np.random.seed(42)
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Generate synthetic container count data for demonstration
container_data = []
for port in ports:
    base_count = np.random.randint(1000, 5000)
    print(f"  • {port['name']:15} - Base count: {base_count:,} containers")
    
    for date in dates:
        # Add trend and noise
        trend = np.sin(len(container_data) * 0.1) * 500
        noise = np.random.normal(0, 200)
        count = max(0, int(base_count + trend + noise))
        
        container_data.append({
            'port': port['name'],
            'timestamp': date,
            'container_count': count
        })

df_containers = pd.DataFrame(container_data)
print(f"\n✓ Generated {len(df_containers):,} container count observations")
print(f"✓ Date range: {dates[0].strftime('%Y-%m-%d')} to {dates[-1].strftime('%Y-%m-%d')}")
print(f"\nFirst 10 observations:")
print(df_containers.head(10))

## Trading Signal Generation

Generate trading signals from container volume analysis:

In [ ]:
# Generate trading signals
signal_gen.load_container_data(df_containers)
port_signals = signal_gen.calculate_signals()
global_signals = signal_gen.generate_global_signal(port_signals)

print("Trading signals generated:")
print(global_signals.tail())

In [ ]:
# Visualization
print("\nGenerating trading signal visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Container counts by port
print("  • Plotting container counts by port...")
for port in ports:
    port_data = df_containers[df_containers['port'] == port['name']]
    axes[0,0].plot(port_data['timestamp'], port_data['container_count'], 
                   label=f"{port['name']} ({port['country']})", marker='o', markersize=3)
axes[0,0].set_title('Container Counts by Port', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Date', fontsize=11)
axes[0,0].set_ylabel('Container Count', fontsize=11)
axes[0,0].legend(loc='best', fontsize=9)
axes[0,0].grid(True, alpha=0.3)
axes[0,0].tick_params(axis='x', rotation=45)

# Global container volume
print("  • Plotting global container volume...")
axes[0,1].plot(global_signals['timestamp'], global_signals['container_count'], 
               color='blue', linewidth=2, marker='o', markersize=4)
axes[0,1].set_title('Global Container Volume (All Ports)', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Date', fontsize=11)
axes[0,1].set_ylabel('Total Container Count', fontsize=11)
axes[0,1].grid(True, alpha=0.3)
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].fill_between(global_signals['timestamp'], global_signals['container_count'], 
                        alpha=0.3, color='blue')

# Trading signals
print("  • Plotting trading signals...")
signal_colors = {1: 'green', -1: 'red', 0: 'gray'}
for signal_val, color in signal_colors.items():
    mask = global_signals['global_signal'] == signal_val
    axes[1,0].scatter(global_signals[mask]['timestamp'], 
                     global_signals[mask]['global_signal'],
                     c=color, s=100, alpha=0.7, 
                     label=f"{'Buy' if signal_val == 1 else 'Sell' if signal_val == -1 else 'Hold'}")
axes[1,0].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1,0].set_title('Global Trading Signal', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Date', fontsize=11)
axes[1,0].set_ylabel('Signal (-1: Sell, 0: Hold, 1: Buy)', fontsize=11)
axes[1,0].set_ylim(-1.5, 1.5)
axes[1,0].grid(True, alpha=0.3)
axes[1,0].legend(loc='best')
axes[1,0].tick_params(axis='x', rotation=45)

# Warning signals
print("  • Plotting warning signals...")
warning_dates = global_signals[global_signals['warning']]['timestamp']
normal_dates = global_signals[~global_signals['warning']]['timestamp']
axes[1,1].scatter(warning_dates, [1]*len(warning_dates), 
                 color='red', s=150, alpha=0.8, marker='X', label='High Volume Warning')
axes[1,1].scatter(normal_dates, [0]*len(normal_dates), 
                 color='green', s=50, alpha=0.5, marker='o', label='Normal Volume')
axes[1,1].set_title('High Volume Warning Signals', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Date', fontsize=11)
axes[1,1].set_ylabel('Status', fontsize=11)
axes[1,1].set_ylim(-0.5, 1.5)
axes[1,1].set_yticks([0, 1])
axes[1,1].set_yticklabels(['Normal', 'Warning'])
axes[1,1].grid(True, alpha=0.3)
axes[1,1].legend(loc='best')
axes[1,1].tick_params(axis='x', rotation=45)

plt.suptitle('Satellite Container Analysis - Trading Dashboard', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n✓ Visualizations complete!")

In [ ]:
# Save results
output_dir = Path('data/results')
output_dir.mkdir(parents=True, exist_ok=True)

# Save container data and signals
df_containers.to_csv(output_dir / 'container_counts.csv', index=False)
port_signals.to_csv(output_dir / 'port_signals.csv', index=False)
global_signals.to_csv(output_dir / 'global_signals.csv', index=False)

print(f"Results saved to {output_dir}")
print(f"\nSummary:")
print(f"- Total observations: {len(df_containers)}")
print(f"- Ports analyzed: {df_containers['port'].nunique()}")
print(f"- Date range: {df_containers['timestamp'].min()} to {df_containers['timestamp'].max()}")
print(f"- Average daily global volume: {global_signals['container_count'].mean():.0f} containers")